# Modelling Data

In [292]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import geopandas as gpd
import h3
from shapely import wkt
from sklearn.linear_model import LinearRegression
import seaborn as sns
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import warnings
warnings.filterwarnings('ignore')

## Creating our panel 

### Read in data 

In [227]:
waze = pd.read_csv('fullclean.csv')
waze.drop(columns=['confidence', 'nThumbsUp', 'country'])
waze['date']= pd.to_datetime(waze['date'])

# date data
soccer = pd.read_csv('soccer.csv')
soccer['date']= pd.to_datetime(soccer['date'])
holiday = pd.read_csv('holiday.csv')
holiday['date']= pd.to_datetime(holiday['date'])
schools = pd.read_csv('schools.csv')

hosp = pd.read_csv('hospitals.csv')
weather = pd.read_csv('RainLevels.csv')[['Date', 'Precipitation']]
weather.columns = ['date', 'precip']
weather['date']= pd.to_datetime(weather['date'])

covid = pd.read_csv('stringency.csv')
covid = covid[['date', 'stringency_index']]
covid = covid[covid.date<='2022-01-01']  # stringency index is at daily level, only look when we have waze data
covid['date']= pd.to_datetime(covid['date'])

### Merging the Data
Caveat: we need to think about how the different hexagon resolutions are impacting this merge. Are there any potential issues being introduced?

In [320]:
# merging stringency and precipitation data by date
df = waze.merge(covid,how='left', on='date')
df = df.merge(weather, how='left', on='date')
df = df.merge(soccer, how='left', on='date')
df = df.merge(holiday, how='left', on='date')

# merging school and hospital data by hexagon
df = df.merge(schools, how='left', on=['h6', 'h7', 'h8', 'h9', 'h10'])
df = df.merge(hosp, how='left', on=['h6', 'h7', 'h8', 'h9', 'h10'])

col_dict = {"Denumire_P": "schools", "Nume": "hospitals", "uuid": "alerts", "stringency_index": "stringency"}

print('The final dataframes will look like...')
dfs = []
for i in [9, 10]:
    cols = ['date','h6', 'h7','h' + str(i) ,'stringency_index', 'precip', 'dayofweek', 'month', 'HomeGame', 'IsHoliday']
    x = df.groupby(cols,as_index=False)[['uuid', 'Denumire_P', 'Nume']].count()
    x = x.rename(columns = col_dict)
    dfs.append(x)
dfs[1]   

The final dataframes will look like...


,date,h6,h7,h10,stringency,precip,dayofweek,month,HomeGame,IsHoliday,alerts,schools,hospitals
0,2020-02-26,861e0b217ffffff,871e0b210ffffff,8a1e0b210d37fff,16.67,5,2,2,0.0,0.0,1,0,0
1,2020-02-26,861e0b217ffffff,871e0b212ffffff,8a1e0b21282ffff,16.67,5,2,2,0.0,0.0,1,0,0
2,2020-02-26,861e0b217ffffff,871e0b214ffffff,8a1e0b214537fff,16.67,5,2,2,0.0,0.0,2,0,0
3,2020-02-26,861e0b217ffffff,871e0b214ffffff,8a1e0b216b4ffff,16.67,5,2,2,0.0,0.0,1,0,0
4,2020-02-26,861e0b217ffffff,871e0b214ffffff,8a1e0b38926ffff,16.67,5,2,2,0.0,0.0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199278,2021-12-31,861e0b3afffffff,871e0b3acffffff,8a1e0b3ac54ffff,52.78,8,4,12,0.0,0.0,2,0,0
199279,2021-12-31,861e0b3afffffff,871e0b3aeffffff,8a1e0b3ae66ffff,52.78,8,4,12,0.0,0.0,1,0,0
199280,2021-12-31,861e0b76fffffff,871e0b769ffffff,8a1e0b769a77fff,52.78,8,4,12,0.0,0.0,2,0,0
199281,2022-01-01,861e0b3afffffff,871e0b3acffffff,8a1e0b3ac187fff,52.78,1,5,1,0.0,0.0,1,0,0


### Create interactions

1. get all polynomial terms for numerical variables
2. add these to controls
3. interact all variables

In [325]:
%%time
panels = []
loop=1
for x in dfs:
    controls = x.iloc[:,4:]
    controls = controls.loc[:, controls.columns != 'alerts']
    num = controls[['stringency', 'precip', 'schools', 'hospitals']]
    cat = controls[['dayofweek', 'month', 'HomeGame', 'IsHoliday', ]]

    # get polynomial terms for numerical variables
    for k in range(len(num.columns)):
        for p in range(3):
            if (p)>0:
                num[str(num.columns[k]) + '^' + str(p+1)] = num[num.columns[k]]**(p+1)
    controls = pd.concat((num, cat), axis=1) # controls including polynomial terms
    print('df ' + str(loop) + ' polynomials are ready')

    # now interact polynomial terms with categorical variables
    interaction = PolynomialFeatures(degree=3, include_bias=False, interaction_only=True)
    controls = pd.DataFrame(interaction.fit_transform(controls),
                                columns=interaction.get_feature_names(input_features=controls.columns))
    panels.append(pd.concat((x.iloc[:,:4],x[['alerts']], controls), axis=1))
    print('df ' + str(loop) + ' is interacted')
    loop+=1

# write to csv   
outputpath = '/Users/catherinehayden/WB/cluj'
d = dict(zip([9,10], panels)) 
for key in d:
    d[key].to_csv(outputpath + '/model' + str(key) + '.csv', index = False)
    print('resolution' + str(key) + 'panel is written to disk')

df 1 polynomials are ready
df 1 is interacted
df 2 polynomials are ready
df 2 is interacted
resolution9panel is written to disk
resolution10panel is written to disk
CPU times: user 1min 57s, sys: 10.1 s, total: 2min 7s
Wall time: 2min 18s


next steps: 
 - calculate distance from each hexagon to the points of interest?
 - control for weather
 - control for weekend/weekday

## How do alerts change with the number of schools near and stringency index at the hexagon-day level? 
For variying hexagon resolutions: 6-10
(As hexagon resolution increases, the area it covers decreases)  
Here we are regressing:  
Alerts(h,t) = Schools(h) + Stringency(t); where h: hexagon, t: time  
Looks like number of schools has a positive coefficient and stringency has a negative coefficient for each h3 resolution. As we expected.  
Magnitude of coefficients steadily decrease as resolution increases.

$Alerts_{h,t} = \alpha + \beta Schools_h + \delta Stringency_t + \varepsilon_{h,t}$

In [121]:
betas = []
for i in range(len(dfs)):
    model = smf.ols(formula='uuid ~ Denumire_P + stringency_index + precip', data=dfs[i]).fit() 
    #predictions = model.predict(df[['Denumire_P', 'stringency_index']]) 
    betas.append(model.params)
    print_model = model.summary()
    print('Hex resolution: ' + str(resolutions[i]))
    print(print_model)

Hex resolution: 6
                            OLS Regression Results                            
Dep. Variable:                   uuid   R-squared:                       0.860
Model:                            OLS   Adj. R-squared:                  0.860
Method:                 Least Squares   F-statistic:                 1.222e+04
Date:                Sat, 21 May 2022   Prob (F-statistic):               0.00
Time:                        14:38:38   Log-Likelihood:                -30672.
No. Observations:                5984   AIC:                         6.135e+04
Df Residuals:                    5980   BIC:                         6.138e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           44

Hex resolution: 10
                            OLS Regression Results                            
Dep. Variable:                   uuid   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                  0.071
Method:                 Least Squares   F-statistic:                     5062.
Date:                Sat, 21 May 2022   Prob (F-statistic):               0.00
Time:                        14:38:38   Log-Likelihood:            -3.4207e+05
No. Observations:              197162   AIC:                         6.842e+05
Df Residuals:                  197158   BIC:                         6.842e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            

In [122]:
betas

[Intercept           44.451394
 Denumire_P           8.419535
 stringency_index    -0.322715
 precip              -0.000336
 dtype: float64,
 Intercept           15.658071
 Denumire_P           6.459335
 stringency_index    -0.093867
 precip              -0.000043
 dtype: float64,
 Intercept           6.686531
 Denumire_P          4.244750
 stringency_index   -0.029974
 precip              0.000028
 dtype: float64,
 Intercept           3.411566
 Denumire_P          1.467573
 stringency_index   -0.010394
 precip              0.000028
 dtype: float64,
 Intercept           1.891958
 Denumire_P          0.585174
 stringency_index   -0.003065
 precip              0.000012
 dtype: float64]